In [1]:
from __future__ import print_function, division

import networkx as nx
from graphviz import Digraph

import torch
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from torch.utils.data import Dataset

from datasets import dynamicDataset

from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [92]:
df = pd.read_csv("../data/mathOverflow/sx-mathoverflow.txt", sep=" ", header=None)
df[3] = pd.to_datetime(df[2], unit='s')
df.columns = ['from', 'to', 'unixTime', 'timestamp']
df.set_index('timestamp', inplace=True)

                     from  to    unixTime
timestamp                                
2009-09-29 02:56:28     1   4  1254192988
2009-09-29 03:24:16     3   4  1254194656
2009-09-29 05:36:52     1   2  1254202612
2009-09-29 14:00:04    25   1  1254232804
2009-09-29 22:26:06    14  16  1254263166


In [2]:
# df = dynamicDataset(path="../data/mathOverflow/sx-mathoverflow.txt")
# print(df.data.head())

                     from  to    unixTime
timestamp                                
2009-09-29 02:56:28     1   4  1254192988
2009-09-29 03:24:16     3   4  1254194656
2009-09-29 05:36:52     1   2  1254202612
2009-09-29 14:00:04    25   1  1254232804
2009-09-29 22:26:06    14  16  1254263166


In [3]:
# Group by particular time value (year, month ,date)
data = df.data.groupby(by=[df.data.index.year, df.data.index.month])

# for name, group in data:
#     print(name)
#     print(group)

In [4]:
# Create a list of all the keys within the groupby
keys = list(data.groups.keys())

# Create a list of all the dataframes within the groupby

dfs = []
for key in keys:
    dfs.append(data.get_group(key))

In [5]:
# Concatenate rows from each timestep
combined = []
combined.append(dfs[0])

for i in range(1, len(dfs)):
    # Segment graph by nodes up to and including that time 
    combined.append(pd.concat([combined[i-1], dfs[i]]))
    
# Validate operation
total = 0
for i in range(len(combined)):
    total += len(dfs[i])
    assert(len(combined[i]) == total)

In [112]:
# Recreate adjacency matrix at each time 

adjacencies = []

for i in tqdm(range(len(combined))):
    adjacency = nx.from_edgelist(combined[i].iloc[:, :2].values, create_using=nx.DiGraph)
    nx.write_adjlist(adjacency,"../data/temp/edgeLists/test"+str(i)+".adjlist")
    #adjacencies.append(nx.to_numpy_matrix(adjacency))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [02:00<00:00,  2.69s/it]


In [82]:
# graph1 = nx.from_numpy_matrix(np.vstack(adjacencies[0]))
# np.random.seed(0)
# plt.figure(figsize=(20, 20))

# nx.draw_networkx(graph1)
# plt.draw() 